# Chapter 4: Object Oriented Programming
## 3. Thread
2019-09-29, 2020-10-04

Content:

- 在教多工序 (thread) 之前，最好先有一個所謂單一執行緒的程式執行
- 在執行環境中關於套件、模組、以及類別的引用
- 有關於多工執行緒
- 有關於 logging 的使用
- 關於time這個模組的使用
- 關於免同時競寫衝突 lock 的使用


本 ipynb 檔的執行情境：
- 有些特殊功能，可以父類別的方式遺傳給需要用的類別；或是說這個需要特殊功能的類別，以繼承的方式，繼承了特殊功能的類別。
- 在這個 ipynb 檔中，我們要展現的是可以同時執行執行緒 (thread) 的功能，它被定義在 thread.Thread 的類別中。
- 為了要能看到不同的 thread 「插時/同時」被執行，我們需要記下當下時間的戳記，所以我們也介紹 logging 的功能。在 logging 過程中，我們可以定義每次登載時，需要提供那些資訊 (特別是執行的時間戳記)
- 針對要 logging 的內容，主要還是仰賴 programmer，在適當時機先預留「引子」，然後，在執行整個被監測的程式過程中，透過調整 loggingConfig() 來選定，什麼程級的警示要進行通報。這的限制是一旦設定後，就要到對個結束後，才能改變設定。
- 「同時執行」的極限是不要彼此干擾。像如果同時有兩個執行緒不受節制都在 console 上列印的話，就有可能會讓 console 上的輸出滲雜在一起。我們將這種不能彼此被干擾的資源稱之為「critical region」
- python 裏有 Lock 的類別，可用來當作管是控「critical region」的警示燈，確保在進入前，critical region 不會另外有人正在使用。

In [ ]:
%cd D:\Google Drive\GettingStartedWithPythonAndRaspberryPi-book_release\Chapter04
%pwd

In [ ]:
%ls

# Inheritance (繼承) 是物件導向程式語言 (OOP) 中重要的概念。
在 python 裏，
- 一個類別可以繼承超過一個的父輩
- 有類似 Java 中的 abstract class，這也是要透過繼承的子輩，實現後才可以使用

## (1) Logging
### 待會要顯示多個 threads 執行的順序，需要一個能記錄「當下」時間的機制
[Python] logging 教學

https://stackoverflow.max-everyday.com/2017/10/python-logging/

## log 的機制：
1. programmer 得先在程式中埋下不同層級的「通報」引子；
   - → logging levels
2. 在執行程式之前，要另外設定 logger，告訴它，針對怎樣層級以上的「通報」要進行記錄；
   - → logging.basicConfig()。每次執行開始後，就不能更動。除非再重頭開始前再變動。
3. 通報時可以輸出到執行的螢幕上，或是寫到某個指名的檔案上。

## *Corey Schafer Python Tutorial*
## Logging Basics - Logging to Files, Setting Levels, and Formatting
https://www.youtube.com/watch?v=-ARI4Cz-awo

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/-ARI4Cz-awo" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

- log: 砍樹 → 記錄
- logging: 記錄活動
- logger: 執行記錄活動的人


## Logging Levels

Level	Numeric value
- CRITICAL	50
- ERROR	40
- WARNING	30
- INFO	20
- DEBUG	10
- NOTSET	0

In [ ]:
'''
log 的機制：
1. programmer 得先在程式中埋下不同層級的「通報」引子；
2. 在執行程式之前，要另外設定 logger，告訴它，針對怎樣層級以上的「通報」要進行記錄；
3. 通報時可以輸出到執行的螢幕上，或是寫到某個指名的檔案上。
'''
import time
import logging
import threading # import thread for python2
'''
※ 以下這個 basicConfig() 設定一次之後，就不能再改了。
細節請參考：https://www.youtube.com/watch?v=-ARI4Cz-awo 
'''
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s (T:%(thread)d):- %(message)s")

In [ ]:
# 一旦設定之後，就不能改了，除非關掉，重新執行
help(logging.basicConfig)

## logging — Logging facility for Python
LogRecord attributes

https://docs.python.org/3/library/logging.html

In [ ]:
'''
我們先輸出到 default 的這個 console 上。
我們設定的格式為：
format="%(asctime)s (T:%(thread)d):- %(message)s"
'''
logging.info("this is a logging test")

In [ ]:
%whos

In [ ]:
logging.warning("this is another message")

In [ ]:
'''
這個 debug 的層級比原先設定的層級還低，所以不會印出來。
'''
logging.debug("this is just an info, not that important")

# (2) Threading：執行緒、線程

1. 我們會用到 thread 通常是用於「同步執行」的情境中。
2. 作法是將想要「被同步」的物件，都讓其繼承 threading.Thread 的類別
3. 然後，將要「同步」的功能，(具體實現)寫在 run() method 中。

https://www.maxlist.xyz/2020/03/15/python-threading/

## *Corey Schafer Python Threading Tutorial*
## Run Code Concurrently Using the Threading Module (2019-09-12)
https://www.youtube.com/watch?v=IEEhzQoKtQU

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/IEEhzQoKtQU" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## 先複習一下 funcrion() 裏面 arg 和 kwarg 的宣告

In [ ]:
'''
先複習 function(*args, **kwargs) 的概念
    - args: positional arguments, considered as tuple
    - kwargs: keyword arguments, considered as dictionary
'''
def checkArgs(*args, **kwargs):
    print(args)
    print(kwargs)
    print(type(args))
    print(type(kwargs))

In [ ]:
checkArgs(1, "p2", name = "chung", age = 23)

In [ ]:
'''
先來看為什麼要用 *args, **kwargs
'''
args = [1, 'p2']
kwargs = {'name': 'chung', 'age': 23}
checkArgs(args, kwargs)

In [ ]:
checkArgs(*args, **kwargs)

### 先看 單一 thread  (Single tread) 執行的狀況：
```python:
class MessagePrinterST(object):
```

In [ ]:
%more Threading.py

In [ ]:
'''
# ST for single thread
因為沒有要用什麼神奇的 tread 功能，所以這邊並沒有繼承 treading.Tread 而是 object

我們要它作的事，定義在 d0_print() 裏。
'''
class MessagePrinterST(object):
    '''
    A simple class to print messages to the log at a given interval.

    以下要作的工作就是將 constructor 建置時的內含的 positional argments 的字串，
    印出來後，會按 kwarg 的 delay 的設定值先睡一下延後。
    '''
    
    def __init__(self, *args, **kwargs):
        self._args = args
        self._kwargs = kwargs

    def do_print(self):
        for message in self._args:
            logging.info(message)
            time.sleep(self._kwargs.get("delay", 1.0))

In [ ]:
%whos

In [ ]:
'''
log 的機制：
1. programmer 得先在程式中埋下不同層級的「通報」引子；
2. 在執行程式之前，要另外設定 logger，告訴它，針對怎樣層級以上的「通報」要進行記錄；
3. 通報時可以輸出到執行的螢幕上，或是寫到某個指名的檔案上。
'''
import time
import logging
import threading # import thread for python2
'''
※ 以下這個 basicConfig() 設定一次之後，就不能再改了。
細節請參考：https://www.youtube.com/watch?v=-ARI4Cz-awo 
'''
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s (T:%(thread)d):- %(message)s")

In [ ]:
# 先建一個沒有 args 的 MessagePrinterST
m1 = MessagePrinterST()

In [ ]:
'''
如果不塞任何參數的話，就不會印東西
'''
m1.do_print()

In [ ]:
# 試在 args 裏放一些 message
m1 = MessagePrinterST("Hello", "Good day!", "Interesting python")

In [ ]:
print(m1._args)

In [ ]:
m1.__dict__.keys()

In [ ]:
m1._args

In [ ]:
m1._kwargs

In [ ]:
# 注意看列印的時間
m1.do_print()

In [ ]:
# 試試 keyward argument，這裏就是 delay
m2 = MessagePrinterST("A", "B", "C", delay=2.0)
m2.do_print()

In [ ]:
m2.__dict__

In [ ]:
m2._kwargs

In [ ]:
'''
以下是 sequential 的執行兩個程序 (不同步執行)
一定要一個執行完了之後，再換另外一個
'''
m1.do_print()
m2.do_print()

### 以下先載入我們的 Threading.py module 檔案

In [ ]:
%more Threading.py

In [ ]:
%whos

In [ ]:
from Threading import *

In [ ]:
%whos

In [ ]:
? MessagePrinter

In [ ]:
??MessagePrinter

In [ ]:
help(MessagePrinter)

In [ ]:
help(threading.Thread)

In [ ]:
threading.Thread?

In [ ]:
'''
觀察重點：與之前 MessagePrinterST(object) 不一樣的地方：
    - 繼承了 threading.Thread
    - 有實現 run()，這是當說要「同步執行」的，要作的事，對應之前的 d0_print()
    
 |  run(self)
 |      Method representing the thread's activity.
 |      
 |      You may override this method in a subclass. The standard run() method
 |      invokes the callable object passed to the object's constructor as the
 |      target argument, if any, with sequential and keyword arguments taken
 |      from the args and kwargs arguments, respectively.
'''

class MessagePrinter(threading.Thread):
    """
    A simple class to print messages to the log a a given interval.
    """
    
    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self)
        self._args = args
        self._kwargs = kwargs

    '''
    當作 thread 最重要的功能，就是要「同步」，同步時要作的工作，
    就是由 run() 來定義。
    '''
    def run(self):
        for message in self._args:
            logging.info(message)
            time.sleep(self._kwargs.get("delay", 1.0))

## thread 最重要的三個 methods:
- start()：所有要同步執行的 thread，以這個為提示，當作：開始一起跑
- run()：一旦同步開始，各顯神通
- join()：這可以標記同時開始同步執行的 tread，最後一個執行完的時間，然後才會再往下面作其他的指令。

In [ ]:
%whos

In [ ]:
# Set up two message printer objects
# mp2 故意 delay 設為 3，(對照 default 的 1) 錯開 logging 的結果
mp1 = MessagePrinter("Hello", "Good day!")
mp2 = MessagePrinter("A", "B", "C", delay=3)

# Start each of them on their own thread
'''
start() 是讓參加「同步」的 thread 一起開始 run()
'''
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
'''
join() 讓所有同步的 thread 一起結束
'''
#mp1.join()
#mp2.join()

logging.info("兩個都作完了")

In [ ]:
# Set up two message printer objects
# mp2 故意 delay 設為 3，(對照 default 的 1) 錯開 logging 的結果
mp1 = MessagePrinter("Hello", "Good day!")
mp2 = MessagePrinter("A", "B", "C", delay=3)

# Start each of them on their own thread
'''
start() 是讓參加「同步」的 thread 一起開始 run()
'''
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
'''
join() 讓所有同步的 thread 一起結束
'''
mp1.join()
mp2.join()

logging.info("兩個都作完了")

In [ ]:
# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!")
mp2 = MessagePrinter("A", "B", "C", delay=0.5)

# Start each of them on their own thread
'''
start() 是讓參加「同步」的 thread 一起開始 run()
'''
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
'''
join() 讓所有同步的 thread 一起結束
'''
mp1.join()
mp2.join()

logging.info("兩個都作完了")

In [ ]:
# 讓 thread 先睡一下，不會浪費 cpu 的時間
help(time.sleep)

In [ ]:
'''
來試一個有三個 thread 同時執行的狀況
'''
m1 = MessagePrinter("Hello", "Good day!")
m2 = MessagePrinter("A", "B", "C", "D", delay = 1.05)
m3 = MessagePrinter("1", "2", "3", "4", delay = 0.95)

m1.start()
m2.start()
m3.start()

m1.join()
m2.join()
m3.join()

logging.info("全都作完了")

### 當然，也可以只有一個 thread，自己 start()，自己 join()

In [ ]:
m1 = MessagePrinter("Hello", "Good day!")
m1.start()
m1.join()
logging.info("都作完了")

In [ ]:
# 執行 thread 其實就是執行 run() 這個 method
m1.run()

logging.info("都作完了")

## synchronized threads 不需要為相同 class 的 instance

In [ ]:
'''
我們寫另外一個也是繼承 threading.Thread 的類別 TimeStamp
'''
class TimeStamp(threading.Thread):
    """
    A simple class to print messages to the log at a given interval.
    """
    
    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self)
        self._args = args
        self._kwargs = kwargs

    '''
    以下的 run() 才是最重要的！
    '''
    def run(self):
        for timetick in range(0,self._kwargs.get("ticks", 10)):
            logging.info(timetick)
            time.sleep(self._kwargs.get("delay", 0.1))

In [ ]:
'''
以下的 script 是來 demo 不同的 thread 可以讓他們「同步」執行
'''
m1 = MessagePrinter("Hello", "Good day!")
t1 = TimeStamp()

m1.start()
t1.start()

m1.join()
t1.join()
logging.info("都作完了")

In [ ]:
m1 = MessagePrinter("Hello", "Good day!")
t1 = TimeStamp(ticks=7)

m1.start()
t1.start()

m1.join()
t1.join()
logging.info("都作完了")

In [ ]:
%whos

In [ ]:
m1.__dict__

In [ ]:
m2.__dict__

In [ ]:
t1.__dict__

## *Corey Schafer Python Threading* - Multithreading Playlist (2017)
https://www.youtube.com/playlist?list=PLGKQkV4guDKEv1DoK4LYdo2ZPLo6cyLbm

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/2ZwuKeL0aHs" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## (3) Lock
Lock 的概念就是：針對一個共享資源，像是寫資料庫或是搶 console，
- 同一個時間只能夠有一個人 (thread) 佔用、進出，
- 為了防止爭奪，所以，設了一個 lock 的機制：所有想要進出管制區的人，要先取得沒有上鎖的(unlocked) 的 lock，才能進入，
- 進入的當下，必須將 lock 給上鎖 (locked)；然後
- 在使用完資源後，使用人要立即將 lock 歸還 (unlocked)，以讓其他人進出管制區。

讓好幾個線程 (執行緒) 同步執行的一個不好的狀況是，萬一他們同時「進出」「更新」共同的資料時，
如第一個要 x = x + 1，另一個要 x = x - 3，要是 x 一開始是 10，前一個正要作，只讀到 10 + 1，正要寫入 11 時，
另一個輪到，讀 x = 10，然後，先 10 - 3 = 7 寫入，接下來換前一個執行，他的印象還是要 10+1，就把 11 寫入 x。

## *Corey Schafer Python Tutorials*
## Threading Beginners Tutorials - Locks - part 5
先講一個事實：這樣的狀況，很難複製…
https://www.youtube.com/watch?v=8BMPW49DadA&list=PLGKQkV4guDKEv1DoK4LYdo2ZPLo6cyLbm&index=7&t=0s

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/8BMPW49DadA" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [2]:
'''
課本附了一個有 lock 的同步程式，
'''
%more Threading_Locks.py

In [3]:
'''
%more Threading_Locks.py
這個 MessagePrinter(threading.Thread) 與之前的版本的差別：
    - 之前是透過 logger 有調節的機制，會錯開列印
    - 現在讓它直接列印在 console 上，這時會有爭奪的狀況發生
    - console 可視為 critical region

※ 這個有 lock 的 class MessagePrinter(threading.Thread):
與之前的 class MessagePrinter(threading.Thread) 最大差別在於 run() 中，
每次進出時要先檢查：

(1) 進入前：
            if self._lock:
                self._lock.acquire()
                
            # 將原本的 logging 改為 print 到 console
            print(message)
(2) 要離開時：
            # If we have a lock object then now release it
            if self._lock:
                self._lock.release()
'''
import logging
import threading
import time

class MessagePrinter(threading.Thread):
    """
    A simple class to print messages to the log a a given interval.
    """
    
    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self)
        self._args = args
        self._kwargs = kwargs
        '''
        以下多了一個 kwarg: lock
        '''
        self._lock = kwargs.get("lock", None)

    def run(self):
        for message in self._args:
            # If we have a lock object then try to acquire the lock first
            if self._lock:
                self._lock.acquire()
                
            # 將原本的 logging 改為 print 到 console
            print(message)

            # If we have a lock object then now release it
            if self._lock:
                self._lock.release()
                
            time.sleep(self._kwargs.get("delay", 1.0))


In [4]:
%whos

Variable         Type      Data/Info
------------------------------------
MessagePrinter   type      <class '__main__.MessagePrinter'>
logging          module    <module 'logging' from 'C<...>b\\logging\\__init__.py'>
threading        module    <module 'threading' from <...>nda3\\lib\\threading.py'>
time             module    <module 'time' (built-in)>


In [5]:
'''
Set up a lock to synchronize the print statements
我們先來看 不設 的狀況
'''
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C", "A", "B", "C","A", "B", "C", lock=lock)


# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

Hello, Good day!A

BHello, Good day!

Hello, Good day!C

Hello, Good day!A

Hello, Good day!B

Hello, Good day!C

AHello, Good day!

B
C


In [6]:
'''
Set up a lock to synchronize the print statements
我們先來看 不設 的狀況
'''
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!","Hello, Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C", "A", "B", "C","A", "B", "C", lock=lock)


# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

Hello, Good day!
A
Hello, Good day!
B
Hello, Good day!
C
Hello, Good day!
A
Hello, Good day!
B
Hello, Good day!
C
Hello, Good day!
A
B
C


## 看你的運氣，還有使用的電腦，真要看到混雜的狀況不容易！

In [47]:
# Set up a lock to synchronize the print statements
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C","A", "B", "C","A", "B", "C","A", "B", "C", lock=lock)


print("{} is used".format(lock))
# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

print("It's done!")

None is used
HelloA

B
Good day!
C
Hello
AGood day!

BHello

Good day!
C
Hello
A
Good day!B

CHello

AGood day!

B
C
It's done!


## 以上是沒有使用 lock 的管控機制

In [7]:
'''
來看 threading.Lock 這個類別
'''
help(threading.Lock)

Help on built-in function allocate_lock in module _thread:

allocate_lock(...)
    allocate_lock() -> lock object
    (allocate() is an obsolete synonym)
    
    Create a new lock object. See help(type(threading.Lock())) for
    information about locks.



In [8]:
type(threading.Lock)

builtin_function_or_method

In [9]:
'''
先來宣告一個 lock 的物件
'''
lock = threading.Lock()

In [10]:
%whos

Variable         Type              Data/Info
--------------------------------------------
MessagePrinter   type              <class '__main__.MessagePrinter'>
lock             lock              <unlocked _thread.lock ob<...>ct at 0x0000019EB8F734B8>
logging          module            <module 'logging' from 'C<...>b\\logging\\__init__.py'>
mp1              MessagePrinter    <MessagePrinter(Thread-8, stopped 7844)>
mp2              MessagePrinter    <MessagePrinter(Thread-9, stopped 10260)>
threading        module            <module 'threading' from <...>nda3\\lib\\threading.py'>
time             module            <module 'time' (built-in)>


In [11]:
type(lock)

_thread.lock

In [12]:
# lock 有兩個狀態：unlocked, locked
lock

<unlocked _thread.lock object at 0x0000019EB8F734B8>

In [13]:
'''
最重要的 methods 是：
    - acquire()
    - release()
它們都會改變 lock 的狀態
'''
help(lock)

Help on lock object:

class lock(builtins.object)
 |  A lock object is a synchronization primitive.  To create a lock,
 |  call threading.Lock().  Methods are:
 |  
 |  acquire() -- lock the lock, possibly blocking until it can be obtained
 |  release() -- unlock of the lock
 |  locked() -- test whether the lock is currently locked
 |  
 |  A lock is not owned by the thread that locked it; another thread may
 |  unlock it.  A thread attempting to lock a lock that it has already locked
 |  will block until another thread unlocks it.  Deadlocks may ensue.
 |  
 |  Methods defined here:
 |  
 |  __enter__(...)
 |      acquire(blocking=True, timeout=-1) -> bool
 |      (acquire_lock() is an obsolete synonym)
 |      
 |      Lock the lock.  Without argument, this blocks if the lock is already
 |      locked (even by the same thread), waiting for another thread to release
 |      the lock, and return True once the lock is acquired.
 |      With an argument, this will only block if the argum

In [14]:
type(lock)

_thread.lock

In [15]:
'''
lock 有兩個狀態，一開始建置時 unlock 的狀況
'''
print(lock)

<unlocked _thread.lock object at 0x0000019EB8F734B8>


In [16]:
lock.acquire()

True

In [17]:
print(lock)

<locked _thread.lock object at 0x0000019EB8F734B8>


In [18]:
lock.release()

In [19]:
print(lock)

<unlocked _thread.lock object at 0x0000019EB8F734B8>


In [20]:
# Set up a lock to synchronize the print statements
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C","A", "B", "C","A", "B", "C", lock=lock)


# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

Hello
A
Good day!
B
Hello
C
Good day!
A
Hello
B
Good day!
C
A
B
C


In [21]:
print(lock)

None


In [22]:
MessagePrinter??

### 如果剛好給你遇上了，會看到竟然這兩個 thread 搶要列印一行 console‧‧‧我以前常看到，但這次試了好多，就是沒看到…

In [47]:
# Set up a lock to synchronize the print statements
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C","A", "B", "C","A", "B", "C","A", "B", "C", lock=lock)


print("{} is used".format(lock))
# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

print("It's done!")

None is used
HelloA

B
Good day!
C
Hello
AGood day!

BHello

Good day!
C
Hello
A
Good day!B

CHello

AGood day!

B
C
It's done!


## 不管你有沒有遇到，要是試久了，就會遇到；
特別是同時要寫到一個檔案時。
以下我們先教要如何「保證完全」避免 critical session 的爭奪，
就是要用的人，要先拿到進出此「關鍵區域」的鑰匙 (lock)

## 在原 MessagePrinter  另外宣告 一個 kwarg變數 lock
除此之外，MessagePrinter 的其他 code 都一樣，
但在進出 critical section時，就要針對管制 console 的 lock, 進行：
- 進入前 aquire() 以及 
- 離開時 release() 的動作。

## 首先，我們先看，如果沒有設鎖的話，會如何…參雜

In [23]:
print(lock)

None


In [24]:
class MessagePrinter(threading.Thread):
    """
    A simple class to print messages to the log a a given interval.
    """
    
    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self)
        self._args = args
        self._kwargs = kwargs
        self._lock = kwargs.get("lock", None)

    def run(self):
        for message in self._args:
            # If we have a lock object then try to acquire the lock first
            if self._lock:
                self._lock.acquire()
                
            print(message)

            # If we have a lock object then now release it
            if self._lock:
                self._lock.release()
            #logging.info(message) #我自己加入這行    
            time.sleep(self._kwargs.get("delay", 1.0))

## Null in Python: Understanding Python's NoneType Object

*Python uses the keyword None to define null objects and variables. While None does serve some of the same purposes as null in other languages, it’s another beast entirely. As the null in Python, None is not defined to be 0 or any other value. In Python, None is an object and a first-class citizen!*

https://realpython.com/null-in-python/

## Python對<type ‘NoneType’>資料型別的處理
https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/634089/

In [25]:
type(None)

NoneType

In [26]:
type("")

str

### None 不等於 False，但是當作 if 條件式判斷時，都是「不成立」的那條分枝…

In [27]:
if False:
    print("None")
else:
    print("hihi")

hihi


In [28]:
# None 在 if 條件式的模查中，等效為 false = null = 沒有，就是不成立
if None:
    print ("None")
else:
    print ("hihi")

hihi


In [29]:
print(None == False)

False


### None 不等於 ""，但是當作 if 條件式判斷時，都是「不成立」的那條分枝…

In [30]:
"" == None

False

In [31]:
if "":
    print ("None")
else:
    print ("hihi")

hihi


In [31]:
%more Threading_Locks.py

In [32]:
import logging
import threading
import time

class MessagePrinter(threading.Thread):
    """
    A simple class to print messages to the log a a given interval.
    """
    
    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self)
        self._args = args
        self._kwargs = kwargs
        self._lock = kwargs.get("lock", None)

    def run(self):
        for message in self._args:
            # If we have a lock object then try to acquire the lock first
            if self._lock:
                self._lock.acquire()
                
            print(message)

            # If we have a lock object then now release it
            if self._lock:
                self._lock.release()
                
            time.sleep(self._kwargs.get("delay", 1.0))


In [33]:
help(threading.Lock)

Help on built-in function allocate_lock in module _thread:

allocate_lock(...)
    allocate_lock() -> lock object
    (allocate() is an obsolete synonym)
    
    Create a new lock object. See help(type(threading.Lock())) for
    information about locks.



In [35]:
help(threading.Lock())

Help on lock object:

class lock(builtins.object)
 |  A lock object is a synchronization primitive.  To create a lock,
 |  call threading.Lock().  Methods are:
 |  
 |  acquire() -- lock the lock, possibly blocking until it can be obtained
 |  release() -- unlock of the lock
 |  locked() -- test whether the lock is currently locked
 |  
 |  A lock is not owned by the thread that locked it; another thread may
 |  unlock it.  A thread attempting to lock a lock that it has already locked
 |  will block until another thread unlocks it.  Deadlocks may ensue.
 |  
 |  Methods defined here:
 |  
 |  __enter__(...)
 |      acquire(blocking=True, timeout=-1) -> bool
 |      (acquire_lock() is an obsolete synonym)
 |      
 |      Lock the lock.  Without argument, this blocks if the lock is already
 |      locked (even by the same thread), waiting for another thread to release
 |      the lock, and return True once the lock is acquired.
 |      With an argument, this will only block if the argum

## Python线程同步机制: Locks, RLocks, Semaphores, Conditions, Events和Queues
http://yoyzhou.github.io/blog/2013/02/28/python-threads-synchronization-locks/

In [34]:
# Set up a lock to synchronize the print statements
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C", delay=3, lock=lock)

# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

print("It's done!")

Hello
A
Good day!
Hello
Good day!
B
C
It's done!


In [35]:
print(lock)

None


In [36]:
lock

### 現在要設 lock 了，就是利用 constructor
threading.Lock()

In [37]:
# Set up a lock to synchronize the print statements
lock = threading.Lock()

In [38]:
print(lock)

<unlocked _thread.lock object at 0x0000019EB8F8F260>


In [39]:
lock == None

False

In [40]:
lock

<unlocked _thread.lock object at 0x0000019EB8F8F260>

In [41]:
# Set up two message printer objects
# lock = threading.Lock()
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C","A", "B", "C", lock=lock)

print("{} is used".format(lock))
# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()
print("It's done!")

<unlocked _thread.lock object at 0x0000019EB8F8F260> is used
Hello
A
Good day!
B
Hello
C
Good day!
A
Hello
B
Good day!
C
Hello
Good day!
It's done!


## 以下好不容易出現！

In [47]:
# Set up a lock to synchronize the print statements
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C","A", "B", "C","A", "B", "C","A", "B", "C", lock=lock)


print("{} is used".format(lock))
# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

print("It's done!")

None is used
HelloA

B
Good day!
C
Hello
AGood day!

BHello

Good day!
C
Hello
A
Good day!B

CHello

AGood day!

B
C
It's done!


In [42]:
# Set up a lock to synchronize the print statements
lock = None #threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C","A", "B", "C","A", "B", "C","A", "B", "C", lock=lock)


print("{} is used".format(lock))
# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

print("It's done!")

None is used
HelloA

B
Good day!
C
Hello
A
Good day!
B
Hello
C
Good day!
AHello

Good day!B

C
Hello
A
Good day!
B
C
It's done!


In [43]:
# Set up a lock to synchronize the print statements
lock = threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinter("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinter("A", "B", "C","A", "B", "C","A", "B", "C","A", "B", "C", lock=lock)


print("{} is used".format(lock))
# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

print("It's done!")

<unlocked _thread.lock object at 0x0000019EB8F8FAF8> is used
Hello
A
Good day!
B
Hello
C
Good day!
A
Hello
B
Good day!
C
Hello
A
Good day!
B
Hello
C
Good day!
A
B
C
It's done!


## 為了要看到 lock 的 aquire() 以及 release() 的狀況。
以下，我們特別加上 logging，每當 thread 要進出 console 時，就讓他從 console 列印出來

In [44]:
'''
我們特別在每次要「進出關鍵區域」時，特別來看 lock 的 aquire() 與 release()
'''
import logging
import threading
import time

class MessagePrinterWithTS(threading.Thread):
    """
    A simple class to print messages to the log a a given interval.
    """
    
    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self)
        self._args = args
        self._kwargs = kwargs
        self._lock = kwargs.get("lock", None)

    def run(self):
        for message in self._args:
            # If we have a lock object then try to acquire the lock first
            
            logging.info("to get the lock") #我自己加入這行 
            
            if self._lock:
                self._lock.acquire()
                
            logging.info("Just got the lock " + str(lock)) #我自己加入這行  
            
            time.sleep(self._kwargs.get("delay", 0.3))
            print(message)

            logging.info(message) #我自己加入這行    
                        
            # If we have a lock object then now release it
            if self._lock:
                self._lock.release()
            
            logging.info("Unlcoked the lock " + str(lock)) #我自己加入這行  
            
            time.sleep(self._kwargs.get("delay", 1.0))

In [45]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s (T:%(thread)d):- %(message)s")

In [46]:
lock = threading.Lock()

# Set up two message printer objects
mp1 = MessagePrinterWithTS("Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!","Hello", "Good day!", lock=lock)
mp2 = MessagePrinterWithTS("A", "B", "C","A", "B", "C","A", "B", "C","A", "B", "C", lock=lock)


print("{} is used".format(lock))

# Start each of them on their own thread
mp1.start()
mp2.start()

# Wait for those threads to exit before exiting the main thread
mp1.join()
mp2.join()

print("It's done!")

2020-10-13 15:29:19,258 (T:6748):- to get the lock
2020-10-13 15:29:19,269 (T:3052):- to get the lock
2020-10-13 15:29:19,270 (T:6748):- Just got the lock <locked _thread.lock object at 0x0000019EB8F8F8F0>


<unlocked _thread.lock object at 0x0000019EB8F8F8F0> is used


2020-10-13 15:29:19,583 (T:6748):- Hello
2020-10-13 15:29:19,585 (T:6748):- Unlcoked the lock <unlocked _thread.lock object at 0x0000019EB8F8F8F0>
2020-10-13 15:29:19,585 (T:3052):- Just got the lock <locked _thread.lock object at 0x0000019EB8F8F8F0>


Hello


2020-10-13 15:29:19,889 (T:3052):- A
2020-10-13 15:29:19,890 (T:3052):- Unlcoked the lock <unlocked _thread.lock object at 0x0000019EB8F8F8F0>


...

A


2020-10-13 15:29:32,657 (T:3052):- to get the lock
2020-10-13 15:29:32,662 (T:3052):- Just got the lock <locked _thread.lock object at 0x0000019EB8F8F8F0>
2020-10-13 15:29:32,965 (T:3052):- B
2020-10-13 15:29:32,966 (T:3052):- Unlcoked the lock <unlocked _thread.lock object at 0x0000019EB8F8F8F0>


B


2020-10-13 15:29:33,968 (T:3052):- to get the lock
2020-10-13 15:29:33,969 (T:3052):- Just got the lock <locked _thread.lock object at 0x0000019EB8F8F8F0>
2020-10-13 15:29:34,270 (T:3052):- C
2020-10-13 15:29:34,270 (T:3052):- Unlcoked the lock <unlocked _thread.lock object at 0x0000019EB8F8F8F0>


C
It's done!
